In [1]:
from dotenv import load_dotenv
load_dotenv()
import logging
from bs4 import BeautifulSoup
import html2text
import httpx
import yaml
import json
from pydantic import Field, BaseModel
from langgraph.graph import MessagesState, StateGraph, START, END
from typing import List
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.types import Send
import operator
from typing import Annotated
from typing import NamedTuple
from utils.fetch_docs import fetch_documents





# Set up the logger
logging.basicConfig(
    level=logging.INFO,  # Set to DEBUG for detailed logs
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        # logging.FileHandler("scraper.log"),  # Log to a file
        logging.StreamHandler()  # Log to console
    ]
)

logger = logging.getLogger(__name__)

agent_architecture_urls = ["https://langchain-ai.github.io/langgraph/tutorials/multi_agent/multi-agent-collaboration",
 "https://langchain-ai.github.io/langgraph/tutorials/multi_agent/agent_supervisor",
 "https://langchain-ai.github.io/langgraph/tutorials/multi_agent/hierarchical_agent_teams",
 "https://langchain-ai.github.io/langgraph/tutorials/plan-and-execute/plan-and-execute",
 "https://langchain-ai.github.io/langgraph/tutorials/self-discover/self-discover",
]


template = """Your job is to get information from a user about what kind of agent they wish to build.

You should get the following information from them:

- What the objective of the agent is
- Various usecases of the agent 
- Some examples of what the agent will be doing (Input and expected output pairs)

If you are not able to discern this info, ask them to clarify! Do not attempt to wildly guess.

After you are able to discern all the information, call the tool AgentInstruction"""

In [2]:
class AgentInstructions(BaseModel):
    """Instructions on how to build the Agent"""
    objective: str = Field(description= "What is the primary objective of the agent")
    usecases: List[str] = Field(description= "What are the various responsibilities of the agent which it needs to fulfill")
    examples : str = Field(description= "What are some examples of the usage of the agent (input query and expected output from the agent) ?")

class ArchEvaluationReport(BaseModel):
    """Class to represent the architecture evaluation report"""
    name: str = Field(description="Name of the agent architecture being evaluated")
    highlights: str = Field(description="Concise summary of the architecture in 5 lines")
    evaluation_score: int = Field(description="evaluation of suitability of the agentic_architecture against user requirements from 1-10, 1 being least relevant to 10 being most relevant")
    justification: str = Field( description="Justification for the score")
    tailored_design: str = Field(description="Tailored design using the architecture")
    
class ArchEvaluationWithUrl(NamedTuple):
    url: str
    report: ArchEvaluationReport

class AgentBuilderState(MessagesState):
    agent_instructions: AgentInstructions = Field("the requirement analysis generated by the model.")
    arch_evaluation_reports: Annotated[List[ArchEvaluationWithUrl], operator.add] = Field("list of agent architectures suggested in map-reduce step")
    best_agent_architecture: ArchEvaluationWithUrl = Field("The agent architecture best suited to above requirements")
    json_code: str = Field("The json code generated")
    python_code: str = Field("The Python code generated")

class ArchitectureEvaluationState(MessagesState):
    agent_instructions: AgentInstructions = Field("the requirement analysis generated by the model.")
    url: str = Field("url of the agent architecture to evaluate against")

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini", streaming=True)

def requirement_analysis_node(state: AgentBuilderState):
    
    llm_with_tool = llm.bind_tools([AgentInstructions])
    response = llm_with_tool.invoke([SystemMessage(content=template)] + state["messages"])
    
      # Construct the final answer from the arguments of the last tool call  
    if len(response.tool_calls) == 0:
        return {"messages": [response]}
    
    agent_instructions = response.tool_calls[0]
    agent_instructions = AgentInstructions(**agent_instructions["args"])
    
    return {"messages": [response], "agent_instructions": agent_instructions}


In [ ]:
ARCH_EVALUATION_PROMPT = PromptTemplate.from_template(
    """
You are tasked with assessing the provided agentic architecture documentation to determine its relevance and applicability to the user requirements outlined below.

Inputs:

- Agentic Architecture Documentation:
{agent_architecture}

- User Requirements:- Objectives: {objective}
- Use Cases: {responsibilities}
- Examples: {examples}


Deliverables:
Your evaluation should be presented in the following structured format:

- Architecture Name:
Provide the name of the agent architecture being assessed.
- Concise Identifier:
Suggest a brief, descriptive name for the architecture that encapsulates its purpose.
- Key Highlights (2-3 Lines):
Identify and summarize the most significant aspects or features of the architecture.
- Feature Summary:
Offer a detailed overview of the architecture's unique elements, functionalities, and capabilities.
- Relevance Score (1-10):
Assign a score based on the alignment between the architecture's features and the user's requirements (1 = minimally relevant, 10 = highly relevant).
- Score Justification:
Provide a clear and concise rationale (5-10 sentences) for the relevance score, highlighting how specific features match—or fail to match—the user's objectives, use cases, and examples.
- Implementation Proposal:
Outline a tailored approach for leveraging the architecture to meet the user’s requirements. Be specific and actionable, addressing how it can fulfill the stated objectives and responsibilities.
"""
)

In [3]:

def architecture_evaluation_map_node(state: AgentBuilderState):
    return [Send("evaluate_against_architecture", {"agent_instructions": state["agent_instructions"], "url": url}) for url in agent_architecture_urls]

def route_state(state: AgentBuilderState):
    messages = state["messages"]
    if isinstance(messages[-1], AIMessage) and messages[-1].tool_calls:
        return "add_tool_message"
    elif not isinstance(messages[-1], HumanMessage):
        return END
    return "requirement_analysis"

def evaluate_against_architecture(state: ArchitectureEvaluationState):
    agent_instructions: AgentInstructions = state["agent_instructions"]
    url: str = state["url"]
    llm_with_structured_output = llm.with_structured_output(ArchEvaluationReport)
    archEvaluationReport: ArchEvaluationReport = llm_with_structured_output.invoke(
        [SystemMessage(content=ARCH_EVALUATION_PROMPT.format(agent_architecture=fetch_documents(url),
                                                             objective=agent_instructions.objective,
                                                             responsibilities=agent_instructions.usecases,
                                                             examples = agent_instructions.examples))])
    
    return {
        "messages": [AIMessage(content=f"Evaluated architecture {url}, arch_name: {archEvaluationReport.name}")],
        "arch_evaluation_reports": [ArchEvaluationWithUrl(url,archEvaluationReport)],
    }

def best_architecture(state: AgentBuilderState):
    """Select the best architecture based on the evaluation reports."""
    # Sort the architectures based on their evaluation scores
    arch_reports : List[ArchEvaluationWithUrl] = state["arch_evaluation_reports"]
    sorted_architectures = sorted(arch_reports, key=lambda x: x.report.evaluation_score , reverse=True)
    
    # Select the best architecture (the first one in the sorted list)
    best_architecture = sorted_architectures[0]
    
    print("found the best architecture")
    
    # Return the best architecture as the output
    return {
        "messages": [AIMessage(content="Best architecture selected!")],
        "best_agent_architecture": best_architecture,
    }

In [4]:

AGENT_KERNEL_PROMPT = PromptTemplate.from_template(
    """
        Task Overview:
        Design a langgraph StateGraph object implementing the {agent_architecture_name} architecture, tailored to fulfill the user requirements defined below

        <Requirements>
        Objectives: {objective}
        usecases: {responsibilities}
        examples: {examples}
        </Requirements>

        Provided Documentation:
        Refer to the documentation for {agent_architecture_name} below to guide your implementation:
        <Documentation for {agent_architecture_name}>
        {agent_architecture}
        </Documentation for {agent_architecture_name}>
        
        Implementation Suggestions:
        Apply the {agent_architecture_name} architecture concepts and align them with the user requirements provided. Suggestions on tailoring the implementation:
        {agent_tailored}
        
        Expected Output:
        Your task is to produce a compiled StateGraph object.

        Guidelines for Code Generation:
        - Accuracy: Avoid hallucinations or speculative assumptions when writing code. Refer exclusively to the provided documentation.
        - Understanding: Thoroughly comprehend the architecture and examples of code.
        - Customization: Generate code tailored specifically to meet the user requirements.

    """)

In [5]:
def agent_kernel_builder(state: AgentBuilderState):
    """Build the agent kernel using the best architecture."""
    best_architecture: ArchEvaluationWithUrl = state["best_agent_architecture"]
    agent_instructions : AgentInstructions = state["agent_instructions"]
    langgraph_glossary_url = "https://langchain-ai.github.io/langgraph/concepts/low_level/"
    agent_architecture_url : str = best_architecture.url
    agent_architecture_report : ArchEvaluationReport = best_architecture.report
    # agent_architecture_report.name
    #agent_architecture_report.highlights
    #agent_architecture_report.justification
    #agent_architecture_report.tailored_design
    
    response =  llm.invoke([SystemMessage(content=AGENT_KERNEL_PROMPT.format(
        objective=agent_instructions.objective,
        responsibilities=agent_instructions.usecases,
        examples = agent_instructions.examples,
        # langgraph_glossary=fetch_documents(langgraph_glossary_url),
        agent_tailored=agent_architecture_report.tailored_design,
        agent_architecture_name = agent_architecture_report.name,
        agent_architecture=fetch_documents(agent_architecture_url)))])
    
    # Return the generated agent kernel as the output
    return {
        "messages": [AIMessage(content="Generated agent kernel code!")],
        "python_code": response.content,
    }

In [6]:

CODE_TO_JSON_PROMPT = PromptTemplate.from_template("""
You are tasked with converting the following stategraph comPilation code into a JSON. 

Contextual documents for understanding code:
{documents}

The Input code is as follows:
{code_snippet}

OUTPUT: Explaination and JSON. Do not include any code blocks. Seperate the JSON and explaination blocks and ensure that there is an explaination for each line of JSON produced but keep the blocks seperated.
Each Output JSON will have a nodes sections containing all the nodes and an edges section

Please follow:
1. Produce the explaination first and then the JSON after it. DO not produce the JSON first. 
2. For any conditional edges, please include all the nodes that the source of a conditional edge can reach as part of the explaination.
3. Any Edge entry in the JSON can only be conditional(mention conditional: true) if the source for that edge acts as a source for multiple edges. If you cannot point to atleast 2 targets for 1 source, then that source will not have any conditional edges
4. A source can have any number of targets. Please write the explaination for each source node to target node edge
5. Please ensure that the JSON starts with __START__ node and __END__ node with the correct edges from and to them
6. Ensure all elements in the nodes sections of the output json contain the following fields: Schema_info, input_schema, output_schema, description, function_name. Please do not return any entries in the nodes without these fields and these fields can't be empty
7. Ensure all elements in the edges sections of the output json contain the following fields: source, target, routing_conditions, conditional. Please do not return any entries in the edges without these fields and they can't be empty
8. Every node should be a part of atleast one edge, Please ensure this is followed
9. Attach the code snippet for each node aswell that. Please extract it from the Input code


Example output JSON for a node:
    "code_node":{{
        "schema_info": /"/"/"CodeWriterState:
      type: TypedDict
      fields:
      - name: user_query
        type: str
      - name: execution_result
        type: str/"/"/",
    "input_schema": "CodeWriterState",
    "output_schema":"RequiremenCodeWriterStatetAnalysisState",
    "description":"This node analyzes the user_query, if the query is to write a code, it will make a tool call to run the proposed code. This node returns command object",
    "function_name": "code_step"
    }}

Example output JSON for an edge:
edge:{{ source: "abc", target: "cde", routing_condition: "if abc made a tool call then go to cde", "conditional": true}}
edge:{{ source: "abc", target: "xyz", routing_condition: "if abc made an interupt to a human then go to xyz", "conditional": true}}
edge:{{ source: "xyz", target: "_END_", routing_condition: "no nodes to go after xyz, we have our final output for this path", "conditional": false}}s         
""")

In [7]:
def code_to_json_node(state: AgentBuilderState):
    """Convert the generated code to JSON."""
    langgraph_glossary_url = "https://langchain-ai.github.io/langgraph/concepts/low_level/"
    json_code_ouptut = llm.invoke([SystemMessage(content=CODE_TO_JSON_PROMPT.format(
        code_snippet=state["python_code"],
        documents = fetch_documents(langgraph_glossary_url),
        ))])
    
    # Return the JSON code as the output
    return {
        "messages": [AIMessage(content="Generated JSON code!")],
        "json_code": json_code_ouptut.content,
    }


In [ ]:

JSON_CODE_COMBINE_PROMPT = PromptTemplate.from_template("""
You are tasked with verifying that the given edges and nodes mentioned in the JSON follow the input code.

Contextual documents for understanding code:
{documents}

The Input code is as follows:
<Input code>
{code_snippet}
</Input code>
                                                        
<JSON>
{node_json}
</JSON>

OUTPUT: JSON

Please ensure that the JSON follows these rules:
1. Any Edge entry in the JSON can only be conditional(mention conditional: true) if the source for that edge acts as a source for multiple edges. If you cannot point to atleast 2 targets for 1 source, then that source will not have any conditional edges
2. A source can have any number of targets.
3. Please ensure that the JSON starts with __START__ node and __END__ node with the correct edges from and to them
4. Ensure all elements in the nodes sections of the output json contain the following fields: Schema_info, input_schema, output_schema, description, function_name. Please do not return any entries in the nodes without these fields and these fields can't be empty
5. Ensure all elements in the edges sections of the output json contain the following fields: source, target, routing_conditions, conditional. Please do not return any entries in the edges without these fields and they can't be empty
6. Every node should be a part of atleast one edge, Please ensure this is followed
7. Attach the code snippet for each node aswell that. Please extract it from the Input code.
8. Final schema of the json is:
<schema>
Example output JSON for a node:
    "code_node":{{
        "id:: <id of node, similar to name>"
        "schema_info": /"/"/"<input state object>
      type: <overall type>
      fields:
      - name: <name of field>
        type: <type>
      - name: <name of field>
        type: <type>/"/"/",
    "input_schema": <input state object>,
    "output_schema":<output state object>,
    "description":<description>,
    "function_name": <name>
    "code": <python_code>
    }}

Example output JSON for an edge:
edge:{{ source: <source_node>, target: <target_node>, routing_condition: <routing_condition>, "conditional": True/False}}
edge:{{ source: <source_node>, target: <target_node>, routing_condition: <routing_condition>, "conditional": True/False}}
edge:{{ source: <sourse_node">, target: <target_node>, routing_condition: <routing_condition>, "conditional": True/False}}        
</schema>                                               
         
1. You can update the JSON but do not update any pre-exisiting field of JSON unless you have an extremely strong justification for it.
2. If they do not follow the code or if there any missing components then please update the JSON with a justification for it. The justification should draw inspiration from the contextual document mentioned.
3. Please ensure that the conditional edges follow the rules mentioned above                       
4. Also please add a "code" entry in each "code_node" entry of the JSON.
5. "code" entry should have the code of the node that the entry in the JSON is trying to represent
6. Do not update any pre-exisiting field of JSON unless you have an extremely strong justification for it
""")

In [ ]:

JSON_CODE_COMBINE_PROMPT = PromptTemplate.from_template("""
You are tasked with verifying and updating the provided JSON that represents the nodes and edges of the langgraph to ensure it is correct with respect to the input code.

Contextual Documents for Understanding Code:
{documents}

Input Code:
<Input code>{code_snippet}</Input code>
JSON:
<JSON>{node_json}</JSON>

OUTPUT: JSON
Your task is divided into two parts:
- Validation: Verify that the JSON adheres to the rules outlined below.
- Correction and Augmentation: If the JSON is incorrect or incomplete, update it with a clear justification for every change, and include the code snippet for each node extracted from the input code.

Rules for Validation and Update:
- Conditional Edges:- An edge can be marked as conditional: true only if its source acts as a source for multiple edges (i.e., at least two targets).
- If the source does not meet this condition, then it cannot have conditional edges.

- Edge Targets:- Each source can have any number of targets. This flexibility must be maintained.

- Start and End Nodes:- The JSON must begin with the __START__ node and conclude with the __END__ node, with correct edges to and from them. Edges into the END node can also be conditional if they meet the above mentioned conditions      

- Node Structure:- Each node entry in the nodes section of the JSON must include the following non-empty fields:- schema_info
- id
- schema_info
- input_schema
- output_schema
- description
- function_name


- Edge Structure:- Each edge entry in the edges section of the JSON must include the following non-empty fields:- source
- target
- routing_conditions
- conditional


- Node-Edge Relationship:- Every node must be part of at least one edge. Ensure this relationship is consistently followed.

- Node Code Snippets:- Attach a code field to every node in the JSON, extracted directly from the input code.

- Schema Requirements:- The final JSON must conform to the schema provided below:


Schema for JSON. Ensure the following schema is followed. No field should be missing for any node or edge:
- Node Example:

{{
  "code_node": {{
    "id" : "<id of the node, similar to name>"
    "schema_info": "<define the class structure of the state of a node>",
    "input_schema": "<input state object>",
    "output_schema": "<output state object>",
    "description": "<description>",
    "function_name": "<function_name>",
    "code": "<python_code>"
  }}
}}


- Edge Example:

{{
  "edge": {{
    "source": "<source_node>",
    "target": "<target_node>",
    "routing_condition": "<routing_condition>",
    "conditional": true/false
  }}
}}


                                            
Key Instructions:
- Do not update any pre-existing field of the JSON unless you have an extremely strong justification for doing so.
- Clearly document the reasoning behind any additions, updates, or modifications to the JSON. Justifications should draw inspiration from the contextual documents mentioned earlier.
- Ensure conditional edges strictly adhere to the rules outlined above.
- Input_Schema and output_schemas can only have value None in JSON for START and END nodes. Please follow this without fail
- Include a code field for each code_node entry, with the exact code that corresponds to the node in the input code.


""")



In [10]:
def json_better_node(state: AgentBuilderState):
    """Add code to the json flow"""
    langgraph_glossary_url = "https://langchain-ai.github.io/langgraph/concepts/low_level/"
    json_code_ouptut = llm.invoke([SystemMessage(content=JSON_CODE_COMBINE_PROMPT.format(
        code_snippet=state["python_code"],
        documents = fetch_documents(langgraph_glossary_url),
        node_json = state["json_code"]
        ))])
    
    # Return the JSON code as the output
    return {
        "messages": [AIMessage(content="Generated updated JSON code!")],
        "json_code": json_code_ouptut.content,
    }


In [11]:
workflow = StateGraph(AgentBuilderState)
workflow.add_node("requirement_analysis", requirement_analysis_node)
workflow.add_node("evaluate_against_architecture", evaluate_against_architecture)
workflow.add_node("best_architecture", best_architecture)
workflow.add_node("agent_kernel_builder", agent_kernel_builder)
workflow.add_node("code_to_json", code_to_json_node)
workflow.add_node("json_update", json_better_node)

@workflow.add_node
def add_tool_message(state: AgentBuilderState):
    
   
    return {
        "messages": [
            ToolMessage(
                content="Requirements generated!",
                tool_call_id=state["messages"][-1].tool_calls[0]["id"],
            )
        ]
    }

workflow.add_edge("json_update", END)
workflow.add_edge("code_to_json", "json_update")
workflow.add_edge("agent_kernel_builder", "code_to_json")
workflow.add_edge("best_architecture", "agent_kernel_builder")
workflow.add_edge("evaluate_against_architecture", "best_architecture")
workflow.add_conditional_edges("add_tool_message", architecture_evaluation_map_node,["evaluate_against_architecture"])
workflow.add_conditional_edges("requirement_analysis", route_state, ["add_tool_message", "requirement_analysis", END])
workflow.add_edge(START, "requirement_analysis")
infograph = workflow.compile()

In [ ]:
import uuid

cached_human_responses = ["hi!", "rag prompt", "1 rag, 2 none, 3 no, 4 no", "red", "q"]
cached_response_index = 0
config = {"configurable": {"thread_id": str(uuid.uuid4())}}
while True:
    try:
        user = input("User (q/Q to quit): ")
    except:
        user = cached_human_responses[cached_response_index]
        cached_response_index += 1
    print(f"User (q/Q to quit): {user}")
    if user in {"q", "Q"}:
        print("AI: Byebye")
        break
    output = None
    for output in infograph.stream(
        {"messages": [HumanMessage(content=user)]}, config=config, stream_mode="updates"
    ):
        last_message = next(iter(output.values()))["messages"][-1]
        last_message.pretty_print()

    if output and "prompt" in output:
        print("Done!")
        

User (q/Q to quit): 


2025-04-29 00:37:28,215 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================

It seems like your message didn't come through. Could you please provide the details about the agent you wish to build? Specifically, I need to know:

1. What is the objective of the agent?
2. What are the various use cases of the agent?
3. Can you provide some examples of what the agent will be doing (input and expected output pairs)?

Feel free to elaborate on any of these points!
User (q/Q to quit): 1. Primary objective is handle customers, resolve their issues, fetch information for them and conflict to resolution 2. Use cases would be updating customer information, providing customers with the data they ask for, asking customers for their requirements and suggesting them products 3. example 1: input: what is the price of product x? output:  price of x is 300$ example 2: input: Please update my address to "delhi" output: Address updated, here are your new user details-> Name: abc, age 24, Address: Del

2025-04-29 00:37:44,324 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  AgentInstructions (call_PJdxwpzU6USyeZRwKhalU1YE)
 Call ID: call_PJdxwpzU6USyeZRwKhalU1YE
  Args:
    objective: Handle customers, resolve their issues, fetch information for them and conflict to resolution
    usecases: ['Updating customer information', 'Providing customers with the data they ask for', 'Asking customers for their requirements', 'Suggesting products to customers']
    examples: Example 1: input: what is the price of product x? output: price of x is 300$; Example 2: input: Please update my address to "delhi" output: Address updated, here are your new user details-> Name: abc, age 24, Address: Delhi; Example 3: input: I am opening up a bakery and need to setup my supply chain and billing output: We have product x that will output your supply chain monitoring and product Y that would automate your billing and accounting along with, I would also suggest product z which helps with

...

================================== Ai Message ==================================

Generated agent kernel code!


2025-04-29 00:38:55,760 - INFO - HTTP Request: GET https://langchain-ai.github.io/langgraph/concepts/low_level/ "HTTP/1.1 200 OK"
2025-04-29 00:38:58,549 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================

Generated JSON code!


2025-04-29 00:39:22,709 - INFO - HTTP Request: GET https://langchain-ai.github.io/langgraph/concepts/low_level/ "HTTP/1.1 200 OK"
2025-04-29 00:39:24,973 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================

Generated updated JSON code!


In [ ]:
{
  "nodes": {
    "__START__": {
      "schema_info": "None",
      "input_schema": "None",
      "output_schema": "None",
      "description": "The entry point of the graph.",
      "function_name": "__START__",
      "code": "__START__"
    },
    "supervisor": {
      "schema_info": "Router: TypedDict with next as a Literal of options",
      "input_schema": "State",
      "output_schema": "Command[Literal[*members, '__end__']]",
      "description": "This node determines which worker to act next based on user input.",
      "function_name": "supervisor_node",
      "code": "def supervisor_node(state: State) -> Command[Literal[*members, '__end__']]:\n    messages = [\n        {\"role\": \"system\", \"content\": system_prompt},\n    ] + state[\"messages\"]\n    response = llm.with_structured_output(Router).invoke(messages)\n    goto = response[\"next\"]\n    if goto == \"FINISH\":\n        goto = END\n    return Command(goto=goto, update={\"next\": goto})"
    },
    "food_logger": {
      "schema_info": "State: MessagesState with next as a string",
      "input_schema": "State",
      "output_schema": "Command[Literal['supervisor']]",
      "description": "This node logs food intake and calculates calories.",
      "function_name": "food_logger_node",
      "code": "def food_logger_node(state: State) -> Command[Literal['supervisor']]:\n    result = food_logger_agent.invoke(state)\n    return Command(\n        update={\n            \"messages\": [\n                HumanMessage(content=result[\"messages\"][-1].content, name=\"food_logger\")\n            ]\n        },\n        goto=\"supervisor\",\n    )"
    },
    "workout_tracker": {
      "schema_info": "State: MessagesState with next as a string",
      "input_schema": "State",
      "output_schema": "Command[Literal['supervisor']]",
      "description": "This node logs workouts and calculates calories burned.",
      "function_name": "workout_tracker_node",
      "code": "def workout_tracker_node(state: State) -> Command[Literal['supervisor']]:\n    result = workout_tracker_agent.invoke(state)\n    return Command(\n        update={\n            \"messages\": [\n                HumanMessage(content=result[\"messages\"][-1].content, name=\"workout_tracker\")\n            ]\n        },\n        goto=\"supervisor\",\n    )"
    },
    "__END__": {
      "schema_info": "None",
      "input_schema": "None",
      "output_schema": "None",
      "description": "The termination point of the graph.",
      "function_name": "__END__",
      "code": "__END__"
    }
  },
  "edges": [
    {
      "source": "__START__",
      "target": "supervisor",
      "routing_conditions": "Start the graph execution by invoking the supervisor node.",
      "conditional": false
    },
    {
      "source": "supervisor",
      "target": "food_logger",
      "routing_conditions": "If the supervisor decides to log food, go to food_logger.",
      "conditional": true
    },
    {
      "source": "supervisor",
      "target": "workout_tracker",
      "routing_conditions": "If the supervisor decides to log workouts, go to workout_tracker.",
      "conditional": true
    },
    {
      "source": "supervisor",
      "target": "__END__",
      "routing_conditions": "If the supervisor decides to finish, go to __END__.",
      "conditional": false
    },
    {
      "source": "food_logger",
      "target": "supervisor",
      "routing_conditions": "After logging food, return to supervisor for next action.",
      "conditional": false
    },
    {
      "source": "workout_tracker",
      "target": "supervisor",
      "routing_conditions": "After logging workouts, return to supervisor for next action.",
      "conditional": false
    }
  ]
}

In [ ]:
{
  "nodes": {
    "__START__": {
      "schema_info": "Type: None",
      "input_schema": "None",
      "output_schema": "None",
      "description": "The starting point of the graph.",
      "function_name": "",
      "code": ""
    },
    "supervisor": {
      "schema_info": "Router: type: TypedDict fields: - name: next type: Literal['food_logger', 'workout_logger', 'FINISH']",
      "input_schema": "MessagesState",
      "output_schema": "Router",
      "description": "This node manages the conversation and routes to the appropriate worker based on user input.",
      "function_name": "supervisor_node",
      "code": "def supervisor_node(state: State) -> Command[Literal[*members, '__end__']]:\n    messages = [\n        {'role': 'system', 'content': system_prompt},\n    ] + state['messages']\n    response = llm.with_structured_output(Router).invoke(messages)\n    goto = response['next']\n    if goto == 'FINISH':\n        goto = END\n\n    return Command(goto=goto, update={'next': goto})"
    },
    "food_logger": {
      "schema_info": "Type: MessagesState",
      "input_schema": "MessagesState",
      "output_schema": "MessagesState",
      "description": "This node logs food items and returns the result to the supervisor.",
      "function_name": "food_logging_node",
      "code": "def food_logging_node(state: State) -> Command[Literal['supervisor']]:\n    food_item = state['messages'][-1]['content']  # Get the last user input\n    result = food_logging_agent_tool.invoke(food_item)\n    return Command(\n        update={\n            'messages': [\n                HumanMessage(content=result, name='food_logger')\n            ]\n        },\n        goto='supervisor',\n    )"
    },
    "workout_logger": {
      "schema_info": "Type: MessagesState",
      "input_schema": "MessagesState",
      "output_schema": "MessagesState",
      "description": "This node logs workout details and returns the result to the supervisor.",
      "function_name": "workout_logging_node",
      "code": "def workout_logging_node(state: State) -> Command[Literal['supervisor']]:\n    workout_details = state['messages'][-1]['content']  # Get the last user input\n    result = workout_analysis_tool.invoke(workout_details)\n    return Command(\n        update={\n            'messages': [\n                HumanMessage(content=result, name='workout_logger')\n            ]\n        },\n        goto='supervisor',\n    )"
    },
    "__END__": {
      "schema_info": "Type: None",
      "input_schema": "None",
      "output_schema": "None",
      "description": "The end point of the graph.",
      "function_name": "",
      "code": ""
    }
  },
  "edges": [
    { "source": "__START__", "target": "supervisor", "routing_condition": "Start the conversation by invoking the supervisor.", "conditional": false },
    { "source": "supervisor", "target": "food_logger", "routing_condition": "If the supervisor routes to food_logger based on user input.", "conditional": true },
    { "source": "supervisor", "target": "workout_logger", "routing_condition": "If the supervisor routes to workout_logger based on user input.", "conditional": true },
    { "source": "supervisor", "target": "__END__", "routing_condition": "If the supervisor routes to FINISH, end the conversation.", "conditional": false },
    { "source": "food_logger", "target": "supervisor", "routing_condition": "After logging food, return to the supervisor.", "conditional": false },
    { "source": "workout_logger", "target": "supervisor", "routing_condition": "After logging workout, return to the supervisor.", "conditional": false }
  ]
}

In [1]:
# {
#   "nodes": {
#     "__START__": {
#       "schema_info": "",
#       "input_schema": "",
#       "output_schema": "",
#       "description": "Entry point of the graph.",
#       "function_name": ""
#     },
#     "planner": {
#       "schema_info": "PlanExecute: TypedDict with fields input (str), plan (List[str]), past_steps (Annotated[List[Tuple], operator.add]), response (str)",
#       "input_schema": "PlanExecute",
#       "output_schema": "Plan",
#       "description": "Generates a step-by-step plan based on user input.",
#       "function_name": "plan_step"
#     },
#     "agent": {
#       "schema_info": "PlanExecute: TypedDict with fields input (str), plan (List[str]), past_steps (Annotated[List[Tuple], operator.add]), response (str)",
#       "input_schema": "PlanExecute",
#       "output_schema": "PlanExecute",
#       "description": "Executes the first step of the plan and updates past steps.",
#       "function_name": "execute_step"
#     },
#     "replan": {
#       "schema_info": "PlanExecute: TypedDict with fields input (str), plan (List[str]), past_steps (Annotated[List[Tuple], operator.add]), response (str)",
#       "input_schema": "PlanExecute",
#       "output_schema": "Union[Response, Plan]",
#       "description": "Updates the plan based on current state and past steps.",
#       "function_name": "replan_step"
#     },
#     "__END__": {
#       "schema_info": "",
#       "input_schema": "",
#       "output_schema": "",
#       "description": "End point of the graph.",
#       "function_name": ""
#     }
#   },
#   "edges": {
#     "edge_1": {
#       "source": "__START__",
#       "target": "planner",
#       "routing_conditions": "Start the planning process.",
#       "conditional": false
#     },
#     "edge_2": {
#       "source": "planner",
#       "target": "agent",
#       "routing_conditions": "After planning, execute the first step.",
#       "conditional": false
#     },
#     "edge_3": {
#       "source": "agent",
#       "target": "replan",
#       "routing_conditions": "After executing a step, check if replanning is needed.",
#       "conditional": false
#     },
#     "edge_4": {
#       "source": "replan",
#       "target": "agent",
#       "routing_conditions": "If no response is generated, continue to agent for further execution.",
#       "conditional": true
#     },
#     "edge_5": {
#       "source": "replan",
#       "target": "__END__",
#       "routing_conditions": "If a response is generated, end the process.",
#       "conditional": true
#     }
#   }
# }

In [ ]:
# {
#   "nodes": {
#     "__START__": {
#       "schema_info": "None",
#       "input_schema": "None",
#       "output_schema": "None",
#       "description": "The starting point of the graph.",
#       "function_name": "start_node"
#     },
#     "supervisor": {
#       "schema_info": "Router: TypedDict with next as a string literal of options.",
#       "input_schema": "State",
#       "output_schema": "State",
#       "description": "This node manages the conversation between workers and decides which worker to route to next.",
#       "function_name": "supervisor_node"
#     },
#     "food_logger": {
#       "schema_info": "State: TypedDict with messages as a list of messages.",
#       "input_schema": "State",
#       "output_schema": "State",
#       "description": "This node logs food intake and calculates calories.",
#       "function_name": "food_logger_node"
#     },
#     "workout_tracker": {
#       "schema_info": "State: TypedDict with messages as a list of messages.",
#       "input_schema": "State",
#       "output_schema": "State",
#       "description": "This node logs workout details and calculates calories burnt.",
#       "function_name": "workout_tracker_node"
#     },
#     "__END__": {
#       "schema_info": "None",
#       "input_schema": "None",
#       "output_schema": "None",
#       "description": "The endpoint of the graph, indicating completion.",
#       "function_name": "end_node"
#     }
#   },
#   "edges": {
#     { 
#       "source": "__START__", 
#       "target": "supervisor", 
#       "routing_conditions": "Start the process by routing to the supervisor node.", 
#       "conditional": false 
#     },
#     { 
#       "source": "supervisor", 
#       "target": "food_logger", 
#       "routing_conditions": "If the supervisor decides to route to food_logger based on user input.", 
#       "conditional": true 
#     },
#     { 
#       "source": "supervisor", 
#       "target": "workout_tracker", 
#       "routing_conditions": "If the supervisor decides to route to workout_tracker based on user input.", 
#       "conditional": true 
#     },
#     { 
#       "source": "supervisor", 
#       "target": "__END__", 
#       "routing_conditions": "If the supervisor decides to finish the process.", 
#       "conditional": true
#     },
#     { 
#       "source": "food_logger", 
#       "target": "supervisor", 
#       "routing_conditions": "After logging food intake, return control to the supervisor.", 
#       "conditional": false 
#     },
#     { 
#       "source": "workout_tracker", 
#       "target": "supervisor", 
#       "routing_conditions": "After logging workout details, return control to the supervisor.", 
#       "conditional": false 
#     }
#   }
# }

In [ ]:
{
  "nodes": {
    "__START__": {
      "schema_info": "",
      "input_schema": "",
      "output_schema": "",
      "description": "Entry point of the graph.",
      "function_name": "__START__",
      "code": ""
    },
    "supervisor": {
      "schema_info": "State: MessagesState",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node manages interactions between the diet and workout teams.",
      "function_name": "teams_supervisor_node",
      "code": "def teams_supervisor_node(state: State) -> Command[Literal[\"diet_logger\"]]: return Command(goto=\"diet_logger\")"
    },
    "diet_logger": {
      "schema_info": "State: MessagesState",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node logs food intake and returns the logged message.",
      "function_name": "diet_logging_node",
      "code": "def diet_logging_node(state: State) -> Command[Literal[\"supervisor\"]]: result = diet_logger_agent.invoke(state) return Command(update={\"messages\": [HumanMessage(content=result[\"messages\"][-1].content, name=\"diet_logger\")]}, goto=\"supervisor\")"
    },
    "workout_logger": {
      "schema_info": "State: MessagesState",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node logs workouts and returns the logged message.",
      "function_name": "workout_logging_node",
      "code": "def workout_logging_node(state: State) -> Command[Literal[\"supervisor\"]]: result = workout_logger_agent.invoke(state) return Command(update={\"messages\": [HumanMessage(content=result[\"messages\"][-1].content, name=\"workout_logger\")]}, goto=\"supervisor\")"
    },
    "diet_supervisor": {
      "schema_info": "State: MessagesState",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node routes to the diet_logger node.",
      "function_name": "diet_supervisor_node",
      "code": "def diet_supervisor_node(state: State) -> Command[Literal[\"diet_logger\"]]: return Command(goto=\"diet_logger\")"
    },
    "workout_supervisor": {
      "schema_info": "State: MessagesState",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node routes to the workout_logger node.",
      "function_name": "workout_supervisor_node",
      "code": "def workout_supervisor_node(state: State) -> Command[Literal[\"workout_logger\"]]: return Command(goto=\"workout_logger\")"
    },
    "diet_team": {
      "schema_info": "State: MessagesState",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node calls the diet team and invokes the diet graph.",
      "function_name": "call_diet_team",
      "code": "def call_diet_team(state: State) -> Command[Literal[\"supervisor\"]]: response = diet_graph.invoke({\"messages\": state[\"messages\"][-1]}) return Command(update={\"messages\": [HumanMessage(content=response[\"messages\"][-1].content, name=\"diet_team\")]}, goto=\"supervisor\")"
    },
    "workout_team": {
      "schema_info": "State: MessagesState",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node calls the workout team and invokes the workout graph.",
      "function_name": "call_workout_team",
      "code": "def call_workout_team(state: State) -> Command[Literal[\"supervisor\"]]: response = workout_graph.invoke({\"messages\": state[\"messages\"][-1]}) return Command(update={\"messages\": [HumanMessage(content=response[\"messages\"][-1].content, name=\"workout_team\")]}, goto=\"supervisor\")"
    },
    "__END__": {
      "schema_info": "",
      "input_schema": "",
      "output_schema": "",
      "description": "Terminal node of the graph.",
      "function_name": "__END__",
      "code": ""
    }
  },
  "edges": [
    { "source": "__START__", "target": "supervisor", "routing_conditions": "Start execution at the supervisor node.", "conditional": false },
    { "source": "supervisor", "target": "diet_supervisor", "routing_conditions": "Route to diet supervisor for diet-related tasks.", "conditional": true },
    { "source": "supervisor", "target": "workout_supervisor", "routing_conditions": "Route to workout supervisor for workout-related tasks.", "conditional": true },
    { "source": "diet_supervisor", "target": "diet_logger", "routing_conditions": "Route to diet logger to log food intake.", "conditional": false },
    { "source": "workout_supervisor", "target": "workout_logger", "routing_conditions": "Route to workout logger to log workouts.", "conditional": false },
    { "source": "diet_team", "target": "supervisor", "routing_conditions": "Return to supervisor after invoking diet graph.", "conditional": false },
    { "source": "workout_team", "target": "supervisor", "routing_conditions": "Return to supervisor after invoking workout graph.", "conditional": false },
    { "source": "diet_logger", "target": "__END__", "routing_conditions": "End process after logging food intake.", "conditional": false },
    { "source": "workout_logger", "target": "__END__", "routing_conditions": "End process after logging workouts.", "conditional": false }
  ]
}

In [ ]:
{
  "nodes": {
    "__START__": {
      "schema_info": "",
      "input_schema": "",
      "output_schema": "",
      "description": "This is the entry point of the graph.",
      "function_name": "start_node"
    },
    "food_logging_node": {
      "schema_info": "MessagesState: type: TypedDict fields: - messages: list[AnyMessage]",
      "input_schema": "MessagesState",
      "output_schema": "MessagesState",
      "description": "This node logs food intake using the food logging agent and updates the messages.",
      "function_name": "food_logging_node"
    },
    "workout_logging_node": {
      "schema_info": "MessagesState: type: TypedDict fields: - messages: list[AnyMessage]",
      "input_schema": "MessagesState",
      "output_schema": "MessagesState",
      "description": "This node tracks workouts using the workout logging agent and updates the messages.",
      "function_name": "workout_logging_node"
    },
    "__END__": {
      "schema_info": "",
      "input_schema": "",
      "output_schema": "",
      "description": "This is the terminal node of the graph.",
      "function_name": "end_node"
    }
  },
  "edges": {
    { "source": "__START__", "target": "food_logging_node", "routing_conditions": "Start the workflow with food logging.", "conditional": false },
    { "source": "food_logging_node", "target": "workout_logging_node", "routing_conditions": "After logging food, transition to workout tracking.", "conditional": true },
    { "source": "workout_logging_node", "target": "__END__", "routing_conditions": "After tracking workouts, conclude the workflow.", "conditional": false }
  }
}

In [ ]:
 {
   "nodes": {
    "__START__": {
      "schema_info": "None",
      "input_schema": "None",
      "output_schema": "None",
      "description": "The entry point of the graph.",
      "function_name": "__START__",
      "code": ""
    },
    "supervisor": {
      "schema_info": "State: MessagesState",
      "input_schema": "State",
      "output_schema": "Command[Literal[*members, '__end__']]",
      "description": "This node determines which agent to invoke next based on user input.",
      "function_name": "supervisor_node",
      "code": "def supervisor_node(state: State) -> Command[Literal[*members, '__end__']]:\n    messages = [\n        {'role': 'system', 'content': system_prompt},\n    ] + state['messages']\n    response = llm.with_structured_output(Router).invoke(messages)\n    goto = response['next']\n    if goto == 'FINISH':\n        goto = END\n    return Command(goto=goto, update={'next': goto})"
    },
    "food_logger": {
      "schema_info": "State: MessagesState",
      "input_schema": "State",
      "output_schema": "Command[Literal['supervisor']]",
      "description": "This node handles food logging tasks and returns to the supervisor.",
      "function_name": "food_logging_node",
      "code": "def food_logging_node(state: State) -> Command[Literal['supervisor']]:\n    result = food_logger_agent.invoke(state)\n    return Command(\n        update={\n            'messages': [\n                HumanMessage(content=result['messages'][-1].content, name='food_logger')\n            ]\n        },\n        goto='supervisor',\n    )"
    },
    "workout_tracker": {
      "schema_info": "State: MessagesState",
      "input_schema": "State",
      "output_schema": "Command[Literal['supervisor']]",
      "description": "This node manages workout tracking tasks and returns to the supervisor.",
      "function_name": "workout_tracking_node",
      "code": "def workout_tracking_node(state: State) -> Command[Literal['supervisor']]:\n    result = workout_tracker_agent.invoke(state)\n    return Command(\n        update={\n            'messages': [\n                HumanMessage(content=result['messages'][-1].content, name='workout_tracker')\n            ]\n        },\n        goto='supervisor',\n    )"
    },
    "__END__": {
      "schema_info": "None",
      "input_schema": "None",
      "output_schema": "None",
      "description": "The termination point of the graph.",
      "function_name": "__END__",
      "code": ""
    }
  },
  "edges": [
    {
      "source": "__START__",
      "target": "supervisor",
      "routing_conditions": "Start the graph execution by invoking the supervisor.",
      "conditional": false
    },
    {
      "source": "supervisor",
      "target": "food_logger",
      "routing_conditions": "If the supervisor decides to log food, go to food_logger.",
      "conditional": true
    },
    {
      "source": "supervisor",
      "target": "workout_tracker",
      "routing_conditions": "If the supervisor decides to track workouts, go to workout_tracker.",
      "conditional": true
    },
    {
      "source": "food_logger",
      "target": "supervisor",
      "routing_conditions": "After logging food, return to the supervisor for further actions.",
      "conditional": false
    },
    {
      "source": "workout_tracker",
      "target": "supervisor",
      "routing_conditions": "After tracking workouts, return to the supervisor for further actions.",
      "conditional": false
    },
    {
      "source": "supervisor",
      "target": "__END__",
      "routing_conditions": "If the supervisor decides to finish, go to the end.",
      "conditional": false
    }
  ]
}

In [ ]:
1. Primary objective is handle customers, resolve their issues, fetch information for them and conflict to resolution 2. Use cases would be updating customer information, providing customers with the data they ask for, asking customers for their requirements and suggesting them products 3. example 1: input: what is the price of product x? output:  price of x is 300$ example 2: input: Please update my address to "delhi" output: Address updated, here are your new user details-> Name: abc, age 24, Address: Delhi Example 3: I am opening up a bakery and need to setup my supply chain and billing output: We have product x that will output your supply chain monitoring and product Y that would automate your billing and accoutning along with, I would also suggest product z which helps with customer out reach and is generally used by customers opening new business and fits your use case

In [ ]:
objective: Help users track their diet and workouts     usecases: ['Track diet', 'Store whatever user eats during a day and track calories', 'Track workouts', 'Provide analysis on optimizing workouts']     examples: Example 1: I ate apple today -> Output: Logged 1 apple as part of breakfast, Calories consumed today: 300. Example 2: I did 4 sets of bicep curls and 2 sets of leg presses -> Output: workout logged, 150 calories burnt during the workout today. You have achieved the weekly goal for bicep training.